In [ ]:
import torch
import functions
import modules
import math
from optimizer import SGD

from torch import optim
from torch import Tensor
from torch import nn

# torch.set_grad_enabled(False)

- Generates a training and a test set of 1,000 points sampled uniformly in [0, 1]2, each with a
label 0 if outside the disk of radius 1/√2π and 1 inside,
- two input units, two output units, three hidden layers of 25 units,

## Test.py

## Generate data

In [ ]:
def generate_disc_set(nb):
    input = torch.Tensor(nb, 2).uniform_(0, 1)
    target = input.pow(2).sum(1).sub(1 / (2 * math.pi)).sign().add(1).div(2).long()
    return input, target

In [ ]:
def convert_to_one_hot_labels(input, target):
    tmp = input.new_zeros(target.size(0), target.max() + 1)
    tmp.scatter_(1, target.view(-1, 1), 1.0)
    return tmp

In [ ]:
train_input, train_target = generate_disc_set(1000)
test_input, test_target = generate_disc_set(1000)
train_target = convert_to_one_hot_labels(train_input, train_target)
test_target = convert_to_one_hot_labels(test_input, test_target)

In [ ]:
train_input = 0.9 * train_input

# Normalization of the data TO DO

In [ ]:
print(train_input.shape)
print(train_target.shape)
print(test_input.shape)
print(test_target.shape)

## Mini-Project Architecture

In [ ]:
model1 = modules.Sequential(modules.Linear(2, 25),
                                  modules.TanH(),
                                  modules.Linear(25, 25),
                                  modules.TanH(),
                                  modules.Linear(25, 25),
                                  modules.TanH(),
                                  modules.Linear(25, 25),
                                  modules.TanH(),
                                  modules.Linear(25, 2),
                                  modules.MSELoss()
                                 )

model2 = modules.Sequential(modules.Linear(2, 25),
                                  modules.ReLU(),
                                  modules.Linear(25, 25),
                                  modules.ReLU(),
                                  modules.Linear(25, 25),
                                  modules.ReLU(),
                                  modules.Linear(25, 25),
                                  modules.ReLU(),
                                  modules.Linear(25, 2),
                                  modules.MSELoss()
                                 )

In [ ]:
nb_epochs = 50
lr = 1e-5
mini_batch_size = 100
optimizer1 = SGD(model1.param(), lr=lr)
optimizer2 = SGD(model2.param(), lr=lr)

In [ ]:
for e in range(nb_epochs):
    sum_loss = 0
    for b in range(0, train_input.size(0), mini_batch_size):
        output, loss = model1.forward(train_input.narrow(0, b, mini_batch_size), train_target.narrow(0, b, mini_batch_size))
        grad = model1.backward()
        sum_loss = sum_loss + loss.item()
        optimizer1.step()
        optimizer1.zero_grad()
    print("Iteration #{0:} loss = {1:.4f}".format(e+1, sum_loss))

In [ ]:
for e in range(nb_epochs):
    sum_loss = 0
    for b in range(0, train_input.size(0), mini_batch_size):
        output, loss = model2.forward(train_input.narrow(0, b, mini_batch_size), train_target.narrow(0, b, mini_batch_size))
        grad = model2.backward()
        sum_loss = sum_loss + loss.item()
        optimizer2.step()
        optimizer2.zero_grad()
    print("Iteration #{0:} loss = {1:.4f}".format(e+1, sum_loss))

In [ ]:
output_test, loss_test = model1.forward(test_input, test_target)

In [ ]:
print(loss_test)